In [1]:
import threading
from time import sleep
import sqlalchemy
import pandas as pd
from sqlalchemy import inspect
from datetime import datetime, timedelta
from tqdm import tqdm

read_engine = sqlalchemy.create_engine('sqlite:////mnt/hgfs/win-d/work/data/20210220-nse.db', echo=False)

In [2]:
tables = read_engine.table_names()
calculate_upto_date = datetime.strptime('18-11-2020', '%d-%m-%Y')

def apply_change_in_oi(df, resampled_df):
    temp = resampled_df[resampled_df['dateTime'] < df.dateTime]
    if temp.empty:
        return 0
    else:
        changeInOi = int(df.openInterest - temp['openInterest'].iloc[-1])
        return  changeInOi

def calculate_change_in_oi(symbol, tables):
    # for idx, table_name in enumerate(tables):
    t = tqdm(enumerate(tables), total=len(tables), desc='Inner Loop')
    for idx, table_name in t:
        try:
            if 'OPTIDX'+symbol not in table_name: continue
            date_idx = len('OPTIDX'+symbol)
            exp_date_str = table_name[date_idx:date_idx+10]
            exp_date = datetime.strptime(exp_date_str, '%d-%m-%Y')

            if exp_date > calculate_upto_date: continue

            t.set_postfix({'table' : table_name})
            # sleep(0.001)
            # print("Processing: {}/{} - {}".format(idx, len(tables), table_name))

            df = pd.read_sql("SELECT * FROM '{}'".format(table_name), con=read_engine)
            df['dateTime'] = pd.to_datetime(df['dateTime'], errors='coerce')
            df.drop_duplicates(keep='first', inplace=True, subset=['dateTime'])

            resampled_df = df.resample('D',on='dateTime').last()
            resampled_df = resampled_df[~resampled_df.dateTime.isnull()]
            # resampled_df = resampled_df.dropna(subset='dateTime')
            df['changeinOpenInterest'] = df.apply(apply_change_in_oi, resampled_df=resampled_df, axis=1)
            df.to_sql(table_name, con=read_engine, if_exists='replace', index=False)
        except Exception as e:
            print(e)
            pass

calculate_change_in_oi('BANKNIFTY', tables)
calculate_change_in_oi('NIFTY', tables)
# t1 = threading.Thread(target=calculate_change_in_oi, args=('BANKNIFTY', tables))
# t1.start()
# # t2 = threading.Thread(target=calculate_change_in_oi, args=('NIFTY', tables))
# # t2.start()
#
# t1.join()
# t2.join()
# calculate_change_in_oi('BANKNIFTY', tables)
# calculate_change_in_oi('NIFTY', tables)

Inner Loop: 100%|██████████| 11647/11647 [1:30:19<00:00,  2.15it/s, table=OPTIDXNIFTY29-10-2020PE9950]   
